# Product Spec

## Set up your `TOGETHER_API_KEY`

In [ ]:
import os

if os.environ.get("TOGETHER_API_KEY") is None:
   os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

## Install Packages to Call the LLM

For this part we set up the LLM.

In [ ]:
!pip3 install openai

In [ ]:
import os
import openai
import textwrap

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(prompt)
    print("")
    print("Response: ")
    print(response)

def get_completion(prompt, temperature=0.0):
    client = openai.OpenAI(api_key=os.environ.get("TOGETHER_API_KEY"),
                           base_url="https://api.together.xyz/v1")
    chat_completion = client.chat.completions.create(
                           model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",
                           #model="zero-one-ai/Yi-34B-Chat",
                           #model="openchat/openchat-3.5-1210",
                           messages=[{"role": "user", "content": prompt}],
                           temperature=temperature,
                           max_tokens=1024)
    response = chat_completion.choices[0].message.content
    response = response.lstrip()
    wrapped_response = wrap_text(response)
    return wrapped_response

## Calling an LLM with `get_completion`

First set up a prompt we can use:

In [ ]:
prompt = "Why is the sky blue? In 50 words or less."

Now we call the`get_completion` function. This function takes our prompt
as argument and sends it off to the LLM. The LLM returns a response.
This function returns that response.

In [ ]:
response = get_completion(prompt)
print(response)